<a href="https://colab.research.google.com/github/vaidasal/ROCT/blob/master/Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [342]:
import json
import pandas as pd
import numpy as np
import plotly.express as px


def readJson(path, type):
    print("Reading Json... ")
    with open(path) as json_file:
        data = json.load(json_file)
        table = pd.read_json(data[type], typ="frame", orient="values")
        df = pd.DataFrame(table)
        df.columns = data[f"C__{type}"]
        return df

Reading JSON

In [343]:
df_original = readJson("/content/drive/MyDrive/100__2021-03-29094210-932382__0.json", "L2")

Reading Json... 


In [344]:
def create1DXCorrected(df, rob_speed):
  v_rob = rob_speed
  # Dataframe aufbereiten
  start_time = df_original.iloc[:,0]
  end_time = df_original.iloc[:,1]
  df = df_original.drop([df_original.columns[0], df_original.columns[1]], axis = 1, inplace = False)
  df = df.replace(r'^\s*$', np.nan, regex=True)
  df = df.apply(pd.to_numeric, errors='ignore')
  cols = pd.to_numeric(df.columns)
  df.columns = np.arange(len(df.columns))

  # Zeitstempel und x-Positionen generieren
  arr_time = np.empty([df.shape[0], cols.size])
  arr_oct_pos = np.empty([df.shape[0], cols.size])
  true_start_time = np.empty(df.shape[0])
  true_end_time = np.empty(df.shape[0])
  for i in range(df.shape[0]):
      first = df.iloc[i].first_valid_index()
      last = df.iloc[i].last_valid_index()
      if first != last:
          timestep = (end_time[i] - start_time[i]) / (last - first)
      true_start_time[i] = start_time[i] - (timestep * first)
      true_end_time[i] = end_time[i] + timestep * (cols.size - 1 - last)
  true_start_time = pd.Series(true_start_time).interpolate()
  true_end_time = pd.Series(true_end_time).interpolate()
  for i in range(df.shape[0]):
      arr_time[i] = np.linspace(true_start_time[i], true_end_time[i], cols.size, endpoint=True)
      arr_oct_pos[i] = cols
  arr_xpos = arr_oct_pos + arr_time * v_rob / 1000

  # Zeitstempel nach x-Position sortieren # Beachte: f_sortiert = f_unsortiert(f_reihenfolge)  <<und>>  f_unsortiert = f_sortiert(f_reihenfolge)
  arr_xpos_1d = arr_xpos.flatten()
  yscatter = df.values.flatten()
  reihenfolge = np.argsort(arr_xpos_1d)
  yscatter_x = np.empty(reihenfolge.size)
  arr_xpos_x = np.empty(reihenfolge.size)
  for i in range(reihenfolge.size):
      yscatter_x[i] = yscatter[reihenfolge[i]]
      arr_xpos_x[i] = arr_xpos_1d[reihenfolge[i]]
    
  return (arr_xpos_x, yscatter_x)

Calculating x,y continuos with min, max values

In [345]:
(x, y) = create1DXCorrected(df_original, 7/60)
xy = pd.Series(y, index=x).dropna()
x_min = np.min(x)
x_max = np.max(x)
df_original_mod = df_original.drop("Line Start [us]", axis=1).drop("Line End [us]", axis=1).reset_index(drop=True).replace("", float("nan")).astype("float64")
y_min = df_original_mod.min().min()
y_max = df_original_mod.max().max()
rows = len(df_original_mod)
print(x_min, x_max, y_min, y_max, rows)

1.1457833333333334 40.916733666666666 -9.714222 1.58193 57


In [346]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'), row=1, col=1)
fig.add_trace(go.Histogram(y=y), row=1, col=2)
fig.show()

Creating Grid

In [367]:
nx, ny = (300, 200)
x_space = np.linspace(x_min, x_max, nx)
y_space = np.flip(np.linspace(y_min, y_max, ny))
df0 = pd.DataFrame(np.zeros((ny,nx)), columns=x_space, index=y_space)

df0.size

60000

Fill Grid

In [368]:
for index, value in xy.items():
  x_sp = (np.abs(x_space-index)).argmin()
  y_sp = (np.abs(y_space-value)).argmin()
  df0.iloc[y_sp, x_sp] = df0.iloc[y_sp, x_sp] + 1

df0

,1.145783,1.278797,1.411810,1.544823,1.677836,1.810849,1.943863,2.076876,2.209889,2.342902,2.475915,2.608929,2.741942,2.874955,3.007968,3.140982,3.273995,3.407008,3.540021,3.673034,3.806048,3.939061,4.072074,4.205087,4.338100,4.471114,4.604127,4.737140,4.870153,5.003166,5.136180,5.269193,5.402206,5.535219,5.668233,5.801246,5.934259,6.067272,6.200285,6.333299,...,35.729218,35.862232,35.995245,36.128258,36.261271,36.394284,36.527298,36.660311,36.793324,36.926337,37.059351,37.192364,37.325377,37.458390,37.591403,37.724417,37.857430,37.990443,38.123456,38.256469,38.389483,38.522496,38.655509,38.788522,38.921535,39.054549,39.187562,39.320575,39.453588,39.586602,39.719615,39.852628,39.985641,40.118654,40.251668,40.384681,40.517694,40.650707,40.783720,40.916734
1.581930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.525165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.468401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.411636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.354872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-9.487164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9.543928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9.600693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9.657457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Dichte Verteilung

In [383]:
from sklearn.cluster import KMeans
sum_entry = np.array(df0.sum(axis=1)).astype(int)
sum_entry

array([   3,   25,    0,    0,    0,    0,    0,    0,    6,    9,    1,
         18,   25,   12,    3,    9,    5,    6,   12,   15,    8,   11,
         27,   29,   23,   17,   39,   84, 3336,   24,   32,   64,   62,
         32,   23,   13,   17,   28,  139,   55,   18,   19,    7,    9,
          5,   11,    6,    7,    4,    9,   11,    6,    3,    5,    4,
          1,    3,    2,    1,    5,    6,   11,    8,    2,   17,   19,
         31,   28,   42,   29,    7,   31,   65,   61,   38,   40,   51,
         61,   29,   20,    8,    8,    9,   12,    5,    3,    1,   12,
         21,    9,   24,    2,    2,    5,    1,    1,    0,    2,    1,
          0,    0,    1,    0,    0,    0,    1,    1,    1,    1,    0,
          0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          2,    2,    1,    1,    1,    0,    2,    0,    0,    0,    2,
          1,    1,    0,    0,    1,    0,    1,    0,    0,    1,    2,
          1,    0,    0,    0,    1,    0,    0,   

Visualisation

In [370]:
fig = px.imshow(df0, color_continuous_scale="Greys", range_color=[0,1])
fig.show()

In [379]:
import copy

def noiseFilter(df_f, row, col, row_len, col_len, size):
  val = df_f.iloc[row,col]

  fil_size = int(size + val)
  r0 = row - fil_size
  r1 = row + fil_size + 1
  if r0 < 0 : r0 = 0
  if r1 > row_len: r1 = row_len
  c0 = col - fil_size
  c1 = col + fil_size + 1
  if c0 < 0 : c0 = 0
  if c1 > col_len: c1 = col_len

  fil_frame = df_f.iloc[r0:r1, c0:c1]

  frame_sum = fil_frame.sum().sum()
  if val == frame_sum: val = 0

  return val

def filterNoise(df0, size):
  df_fil = copy.deepcopy(df0)
  x_len = len(x)
  df_max = df_fil.max().max()
  col_len = len(df_fil.columns)
  row_len = len(df_fil.index)
  col_range = range(col_len)
  row_range = range(row_len)
  for row in row_range:
    for col in col_range:
      if df_fil.iloc[row,col] == 0:
        pass
      else:
        df_fil.iloc[row,col] = noiseFilter(df_fil, row, col, row_len, col_len, size)

  for index, row in df_fil.iloc[::-1].iterrows():
    if row.sum() == 0:
      df_fil.drop(index=index, inplace=True)
    else:
      break
  return df_fil

data = filterNoise(df0, 0)
data = filterNoise(data, -1)

fig = px.imshow(data, color_continuous_scale="Greys", range_color=[0,1])
fig.show()

In [397]:
fig = px.scatter(x=data.sum().index, y=data.sum().values)
fig.show()

In [398]:
data.sum().values.mean()

15.25